In [ ]:
pip install crewai==0.80.0


In [ ]:
import crewai

In [ ]:
crewai.__version__

In [1]:
from dotenv import load_dotenv

In [ ]:
load_dotenv('/home/santhosh/Projects/courses/Pioneer/.env')

In [6]:
from crewai import Agent, Task, Crew, LLM

In [7]:
llm = LLM(model="gpt-4o-mini")

# Recipe Recommendation

In [6]:
# User inputs
main_ingredient = "tomato"
dietary_restrictions = "shrimps"

In [6]:
culinary_assistant = Agent(
    llm=llm,
    role="Culinary Assistant",
    backstory="An experienced culinary assistant skilled in finding and tailoring recipes based on ingredients and dietary"
    "needs, and providing clear, step-by-step cooking instructions",
    goal="Find recipes, filter them to meet dietary preferences, and guide user through recipe steps.",
    verbose=True,
)

In [7]:
find_and_filter_recipes = Task(
    description=f"Find recipes that user the ingredient: {main_ingredient} and"
    f" filter them to meet dietary restrictions: {dietary_restrictions}.",
    expected_output=f"One recipe using {main_ingredient} and matching {dietary_restrictions} restrictions.",
    agent=culinary_assistant,
)

guide_recipe_steps = Task(
    description="Provide step-by-step instructions for the selected recipe.",
    expected_output="Step-by-step cooking instructions for the chosen recipe.",
    agent=culinary_assistant,
)


In [8]:
crew = Crew(
    agents=[culinary_assistant],
    tasks=[find_and_filter_recipes, guide_recipe_steps],
    planning=True,
)

In [ ]:
crew.kickoff()

In [ ]:
crew.usage_metrics

# Marketing Research

In [8]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

In [9]:
product_name = "energy drink"

In [10]:
strategist_backstory = "and marketing strategy"

In [11]:
market_researcher = Agent(
    role="Market Researcher",
    goal="Analyze market trends for the product launch",
    backstory="Experienced in market trends and consumer behavior analysis",
    tools=[SerperDevTool()],
    verbose=True,
)

strategist = Agent(
    role="Product Strategist",
    goal="Create effective positioning strategies for the product",
    backstory=f"Skilled in competitive positioning {strategist_backstory}",
    verbose=True,
)

In [12]:
gather_market_insights_task = Task(
    description=f"Browse the internet to gather insights on current market trends for the launch of the {product_name} product.",
    expected_output=f"List of relevant market trends and consumer preferences, relevant to {product_name}",
    agent=market_researcher,
)

develop_positioning_strategy_task = Task(
    description="Based on the market insights,"
                f"create a positioning strategy for the {product_name} product, including analysis for impact and target audience.",
    expected_output="A positioning strategy with target audience and impact notes",
    agent=strategist,
    verbose=True,
)

In [13]:
crew = Crew(
    agents=[market_researcher, strategist],
    tasks=[gather_market_insights_task, develop_positioning_strategy_task],
    planning=True,
)

In [ ]:
output = crew.kickoff()

In [ ]:
print(output.raw)

In [ ]:
output.token_usage

# Game Flow

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
import random
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

In [9]:
class GameSession(BaseModel):
    player_won: bool = False


class GameFlow(Flow[GameSession]):

    @start()
    def begin_start(self):
        print("Starting the game session")
        player_outcome = random.choice([True, False])
        self.state.player_won = player_outcome

    @router(begin_start)
    def check_outcome(self):
        if self.state.player_won:
            return "win"
        else:
            return "lose"

    @listen("win")
    def celebrate_win(self):
        print("Congratulations!")

    @listen("lose")
    def console_loss(self):
        print("Game Over!")

In [10]:
flow = GameFlow()

In [ ]:
flow.kickoff()